In [1]:
# Imports
import os
import sys
import pandas as pd
import numpy as np
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from matplotlib import pyplot as plt
import seaborn as sns

sns.set_style('dark')

In [2]:
# Get the username from terminal
username = sys.argv[1]

os.environ['SPOTIPY_CLIENT_ID'] = '509f1c0827f94cd7af9c842beb26e7ef'
os.environ['SPOTIPY_CLIENT_SECRET'] = '83fe375561354e6485bde0e952dc5d70'
os.environ['SPOTIPY_REDIRECT_URI'] = 'http://localhost:8888/callback/'

cid = '509f1c0827f94cd7af9c842beb26e7ef' # Client ID
secret = '83fe375561354e6485bde0e952dc5d70'   # Client Secret ID
username = '21ugtw76ynask7ohvvz626ndq' # Spotify Username

scope = 'user-library-read playlist-modify-public playlist-read-private'
redirect_uri = 'http://localhost:8888/'
#http://localhost:8888/callback/

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
token = util.prompt_for_user_token(username, scope, cid, secret, redirect_uri)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [3]:
genres = ['house', 'techno', 'dnb', 'hardstyle', 'trap']

df = pd.read_csv(r'C:\Users\Chetan\Desktop\College\Python\EDM Sub-genre classifier\playlists.csv')
df.head()

,username,playlistid,genre,title,url
0,Élite,5sPzNVd9MbUwmWd2mA90qB,house,Boiler Room | Techno House,https://open.spotify.com/playlist/5sPzNVd9MbUw...
1,Beatport,6Pbk82RPOKWD7k6dNcqq9f,house,Beatport Best New Tech House,https://open.spotify.com/playlist/6Pbk82RPOKWD...
2,Mac Peterson,7AUqarp6W3pf0tVFrRvzZw,house,Dirtybird Players,https://open.spotify.com/playlist/7AUqarp6W3pf...
3,Alex Meyer,1NQK1mGamaJmmwNE4iv9PT,house,House/Dirtybird,https://open.spotify.com/playlist/1NQK1mGamaJm...
4,Dave Lloyd,31QIVbCj6M2KkVxMepKS7b,house,Dirtybird//Techhouse by Mack,https://open.spotify.com/playlist/31QIVbCj6M2K...


In [4]:
def get_features(username, playlist_id, genre):
    results = sp.user_playlist_tracks(username, playlist_id)
    songs = results['items']
    
    while results['next']:
        results = sp.next(results)
        songs.extend(results['items'])
    
    ids = []
        
    for i in range(len(songs)):
        ids.append(songs[i]['track']['id'])
    
    ids = list(filter(None, ids))
    
    features = []
    
    for i in range(0,len(ids),50):
        audio_features = sp.audio_features(ids[i:i+50])
        
        for track in audio_features:
            features.append(track)

    features = list(filter(None, features))
    df = pd.DataFrame(features)
    df['genre'] = genre
    return df


In [5]:
def create_df(PlayLists):
    result = pd.DataFrame()
    
    # Extract songs from spotify API and append to dataframe
    for i in range(len(PlayLists)):
        df = get_features(PlayLists['username'][i], PlayLists['playlistid'][i], PlayLists['genre'][i])
        df['title'] = PlayLists['title'][i]
        result = pd.concat([result, df], ignore_index=True)
    
    return result

In [6]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframelimit=2000
    playlist_features_list = ["artist","album","track_name",  "track_id","danceability","energy","key","loudness","mode", "speechiness","instrumentalness","liveness","valence","tempo", "duration_ms","time_signature"]
    
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Create empty dict
        playlist_features = {}
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [7]:
# These Functions can read more than 100 songs a playlist

def show_tracks(results, uriArray):
    for i, item in enumerate(results['items']):
        track = item['track']
        uriArray.append(track['id'])

def get_playlist_track_id(username, playlist_id):
    trackID = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks, trackID)
    while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks, trackID)
    return trackID

In [8]:
songs_all = []
for i in range (0, 2): #Put Length of the playlist wala list here
    songs = get_playlist_track_id(df['username'][i], df['playlistid'][i])
    songs_all.extend(songs)

In [9]:
len(songs_all)
# Correct list length for all playlist indexed 0,1,2

206

In [26]:
for i in range(0, 10):
    print(sp.audio_features(tracks = songs_all[i]))

[{'danceability': 0.786, 'energy': 0.87, 'key': 2, 'loudness': -5.839, 'mode': 1, 'speechiness': 0.0507, 'acousticness': 0.0493, 'instrumentalness': 0.215, 'liveness': 0.0863, 'valence': 0.676, 'tempo': 126.011, 'type': 'audio_features', 'id': '7LVEBvmKB5uDTg0CGqDo3p', 'uri': 'spotify:track:7LVEBvmKB5uDTg0CGqDo3p', 'track_href': 'https://api.spotify.com/v1/tracks/7LVEBvmKB5uDTg0CGqDo3p', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7LVEBvmKB5uDTg0CGqDo3p', 'duration_ms': 192482, 'time_signature': 4}]
[{'danceability': 0.74, 'energy': 0.934, 'key': 7, 'loudness': -3.255, 'mode': 1, 'speechiness': 0.316, 'acousticness': 0.00822, 'instrumentalness': 0.000377, 'liveness': 0.116, 'valence': 0.685, 'tempo': 124.936, 'type': 'audio_features', 'id': '0m69QNuZVZDXhGHBGK1mQA', 'uri': 'spotify:track:0m69QNuZVZDXhGHBGK1mQA', 'track_href': 'https://api.spotify.com/v1/tracks/0m69QNuZVZDXhGHBGK1mQA', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0m69QNuZVZDXhGHBGK1mQA', 'du

None


In [10]:
def song_feature_ext(playlist):
    result = pd.DataFrame()
    for i in range(len(playlist)):
        song_feat = sp.audio_features(tracks = playlist[i])
        result = pd.concat([result, song_feat], ignore_index=True)

    return result

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [13]:
result = pd.DataFrame()

for i in range(len(songs_all)):
    df = sp.audio_features(songs_all[i])
    result = pd.concat([result, df], ignore_index=True)

result = result.drop_duplicates()
return result

TypeError: cannot concatenate object of type '<class 'list'>'; only Series and DataFrame objs are valid

In [66]:
for i in range(0, len(songs_all)):
    pd.DataFrame(sp.audio_features(songs_all[i]))


KeyError: 0

In [65]:
songs_sahi.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.84,0.868,11,-6.173,0,0.0721,0.0259,0.573,0.0891,0.851,126.001,audio_features,6uABa7vKOgFqXnwvk8NBr5,spotify:track:6uABa7vKOgFqXnwvk8NBr5,https://api.spotify.com/v1/tracks/6uABa7vKOgFq...,https://api.spotify.com/v1/audio-analysis/6uAB...,203810,4


def get_playlist_track_id1(username, playlist_id, genre):
    trackID = []
    results = sp.user_playlist(username, playlist_id)
    tracks = results['tracks']
    show_tracks(tracks, trackID)
    while tracks['next']:
            tracks = sp.next(tracks)
            show_tracks(tracks, trackID)
    return trackID, genre

In [26]:
sp.audio_features(tracks=['6uABa7vKOgFqXnwvk8NBr5'])

[{'danceability': 0.84,
  'energy': 0.868,
  'key': 11,
  'loudness': -6.173,
  'mode': 0,
  'speechiness': 0.0721,
  'acousticness': 0.0259,
  'instrumentalness': 0.573,
  'liveness': 0.0891,
  'valence': 0.851,
  'tempo': 126.001,
  'type': 'audio_features',
  'id': '6uABa7vKOgFqXnwvk8NBr5',
  'uri': 'spotify:track:6uABa7vKOgFqXnwvk8NBr5',
  'track_href': 'https://api.spotify.com/v1/tracks/6uABa7vKOgFqXnwvk8NBr5',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6uABa7vKOgFqXnwvk8NBr5',
  'duration_ms': 203810,
  'time_signature': 4}]

In [39]:
sp.audio_features(tracks = get_playlist_track_id(df['username'][0], df['playlistid'][0]))

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5biCiD9a2EKCVp8L8O8wPs,6UX4Asr5w2q3RVwYTF0wC1,3TRihEWxd7tLwwuZTvjVdA,0dYaNFpo01kKWBnzV4MbGF,6gUDi7QvqriYOYMU2YH0qo,0LO7LNwJ4tHJcv7CuX9uZa,6fslel5Y3JPyeRFSfuAjGe,1OgVa6BCejtHJDTQG6XO6T,7hJ5NZDl0IpsrccbapRpaQ,3CNSqItZGRJSvLEgH1zxr5,6kMVeMGoByxIuWg2BVRXMz,2BD8xyv5o6UpJyAAvoXMPZ,12tUPXD0Vnb8uHTxaVZQfU,2iGulZBZ16s7vGErz9DFJd,0m69QNuZVZDXhGHBGK1mQA,3AjIkDomJhIC4xhyaw66ok,54IysiLnjjhTN7H8X2K9mw,526nTDjQS3dIR0MYyD4gWr,2igAp74O4rtV540H9hMnyQ,7jaA5mg13MhUdFo6LkLreP,7cj4l1zWQI0rDpPSybBuRA,2Cu4UJ2qZvOCFd8XTjjJgx,0jE2RmpcBrh0YNZfp8ceBA,5XztR7gHO7bpK3yXdfV7nk,41274O5qdcalyWv1eBCTl2,4bbS88iLkFjEvRK5FxOtNa,0NX3Jo8fWX20zwXPwxLLAR,37Fnq3qWk110e3uGevD8R9,3YmWaLRgbzZsLkr6zXKB6w,3N28EvZSo2PckQSgg7zgxg,71eu25KcQakQx9TXqiS1NF,0Z0VMtV6tPSY0ww9PL3MdM,6pQl6f19p2zg0M9TpbhM0U,07vHbooCj3zhaa2UIDYj3v,2mFAODPbtZNLi9698SuDvE,6cEgUxJRdS9R21fsOhgs8j,4mZSUkVwYbfQPYiSSNuNb4,3IDc4RYTQ2h30OyRB2BYrV,4YvhiUuNUvXEJsbMLWY5pQ,1S4hGBm48Gbq7fz45k1S4b,2j5yoES74sx

SpotifyException: http status: 400, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5biCiD9a2EKCVp8L8O8wPs,6UX4Asr5w2q3RVwYTF0wC1,3TRihEWxd7tLwwuZTvjVdA,0dYaNFpo01kKWBnzV4MbGF,6gUDi7QvqriYOYMU2YH0qo,0LO7LNwJ4tHJcv7CuX9uZa,6fslel5Y3JPyeRFSfuAjGe,1OgVa6BCejtHJDTQG6XO6T,7hJ5NZDl0IpsrccbapRpaQ,3CNSqItZGRJSvLEgH1zxr5,6kMVeMGoByxIuWg2BVRXMz,2BD8xyv5o6UpJyAAvoXMPZ,12tUPXD0Vnb8uHTxaVZQfU,2iGulZBZ16s7vGErz9DFJd,0m69QNuZVZDXhGHBGK1mQA,3AjIkDomJhIC4xhyaw66ok,54IysiLnjjhTN7H8X2K9mw,526nTDjQS3dIR0MYyD4gWr,2igAp74O4rtV540H9hMnyQ,7jaA5mg13MhUdFo6LkLreP,7cj4l1zWQI0rDpPSybBuRA,2Cu4UJ2qZvOCFd8XTjjJgx,0jE2RmpcBrh0YNZfp8ceBA,5XztR7gHO7bpK3yXdfV7nk,41274O5qdcalyWv1eBCTl2,4bbS88iLkFjEvRK5FxOtNa,0NX3Jo8fWX20zwXPwxLLAR,37Fnq3qWk110e3uGevD8R9,3YmWaLRgbzZsLkr6zXKB6w,3N28EvZSo2PckQSgg7zgxg,71eu25KcQakQx9TXqiS1NF,0Z0VMtV6tPSY0ww9PL3MdM,6pQl6f19p2zg0M9TpbhM0U,07vHbooCj3zhaa2UIDYj3v,2mFAODPbtZNLi9698SuDvE,6cEgUxJRdS9R21fsOhgs8j,4mZSUkVwYbfQPYiSSNuNb4,3IDc4RYTQ2h30OyRB2BYrV,4YvhiUuNUvXEJsbMLWY5pQ,1S4hGBm48G

In [ ]:
songs_all

In [23]:
song_feat1 = sp.audio_features(tracks = '6uABa7vKOgFqXnwvk8NBr5')